# *1.Conexion al drive*

In [ ]:
try:
  from google.colab import drive
  drive.mount('/content/drive')
  print("Drive montado correctamente.")
except ModuleNotFoundError:
  print("No estás en Colab, omitiendo montaje de Drive.")

# *2.Cargar funciones auxiliares desde otro notebook*

In [ ]:
%run "/content/drive/MyDrive/UdeA/Salud/funciones.py"

#*3. Librerias*

In [ ]:
# --- Importar Librerias ---
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import numpy as np
from IPython.display import display
import scipy.stats as stats

#*4. Carga del dataset desde archivo CSV*

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/UdeA/Salud/data/mental_health_dataset.csv')
print("Archivo cargado correctamente")

# *5.Cargar datos*

In [ ]:
def cargar_datos():

  """
    Carga un archivo CSV y retorna un DataFrame.
    Args:
        ruta (str): Ruta al archivo CSV.
    Returns:
        DataFrame de pandas.
  """

  # Crear carpeta dentro de Drive para el proyecto (si no existe)
  project_path = '/content/drive/MyDrive/UdeA/Salud/data'
  os.makedirs(project_path, exist_ok=True)

  # --- 2. Cargar el archivo CSV ----
  # Define la ruta del archivo CSV
  csv_file_path = os.path.join(project_path, 'mental_health_dataset.csv')

  # Carga el archivo CSV en un DataFrame de pandas
  try:
    df = pd.read_csv(csv_file_path)
    print("Archivo CSV cargado correctamente.")
    # Muestra las primeras filas del DataFrame para verificar
  except FileNotFoundError:
    print(f"Error: El archivo CSV no se encontró en la ruta: {csv_file_path}")
  except Exception as e:
    print(f"Ocurrió un error al cargar el archivo CSV: {e}")

# *6.Analisis exploratorio*

In [ ]:
def analisis_exploratorio(df):

  """
    Muestra resumen básico del DataFrame.
  """

  # --- 3. Análisis Exploratorio ---
  print("\n--- Exploración básica del dataset ---")
  print("Primeras 5 filas:")
  display(df.head())

  print("\nInformación general:")
  df.info()

  print("\nDescripción estadística:")
  display(df.describe())

  print("\nDescripción estadística de variables categóricas:")
  display(df.describe(include='object'))

  print("\nConteo de valores nulos por columna:")
  display(df.isnull().sum())

# *7. Visualización de datos*


# *7.1 Variables Numéricas*

### *7.1.1 Histograma*

In [ ]:
def visualizar_histograma(df):

  # Selecciona solo las columnas numéricas
  num_cols = df.select_dtypes(include=['int64', 'float64']).columns

  # Verifica si hay columnas numéricas para graficar
  if len(num_cols) > 0:
      print("\n--- Histogramas para variables numéricas ---")
      df[num_cols].hist(bins=20, figsize=(15, 10))  # Ajusta bins y figsize según necesites
      plt.suptitle('Histogramas de Variables Numéricas', y=1.02)
      plt.tight_layout()
      plt.show()
  else:
      print("No hay columnas de tipo int o float para generar histogramas.")

In [ ]:
# Llama a la función de visualización con tu DataFrame
visualizar_histograma(df)

### *7.1.2 Outliers en variables numéricas*

In [ ]:
def visualizar_boxplots(df):
    """
    Muestra boxplots para identificar outliers en variables numéricas.
    """
    num_cols = df.select_dtypes(include=['int64', 'float64']).columns
    if len(num_cols) > 0:
        num_vars = len(num_cols)
        cols = 3
        rows = (num_vars // cols) + (num_vars % cols > 0)
        fig, axes = plt.subplots(rows, cols, figsize=(15, 5 * rows))
        axes = axes.flatten()
        for i, col in enumerate(num_cols):
            sns.boxplot(x=df[col], ax=axes[i])
            axes[i].set_title(f'Boxplot de {col}')
            axes[i].set_xlabel(col)
        for j in range(len(num_cols), len(axes)):
            axes[j].axis('off')
        plt.tight_layout()
        plt.show()
    else:
        print("No hay columnas numéricas para generar boxplots.")

In [ ]:
def mostrar_qqplots(df):
    """
    Genera QQ-plots para verificar la normalidad de las variables numéricas.
    """
    num_cols = df.select_dtypes(include=['int64', 'float64']).columns
    for col in num_cols:
        plt.figure()
        stats.probplot(df[col].dropna(), dist="norm", plot=plt)
        plt.title(f'QQ-plot para {col}')
        plt.show()

### *7.1.3 Matriz de correlación*

In [ ]:
def correlacion(df):
    """
    Calcula y visualiza la matriz de correlación entre variables numéricas.
    """
    num_cols = df.select_dtypes(include=['int64', 'float64']).columns
    corr_matrix = df[num_cols].corr()
    plt.figure(figsize=(10, 8))
    sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", fmt='.2f', linewidths=0.5)
    plt.title('Matriz de Correlación')
    plt.show()

In [ ]:
correlacion(df)

## *7.2 Variables Categoricas*

### *7.2.1 Resumen de las variables categoricas*

In [ ]:
def resumen_variables_categoricas(df):
    """
    Imprime el número de categorías únicas y las 10 más frecuentes de cada variable categórica.
    """
    cat_cols = df.select_dtypes(include='object').columns.tolist()
    for col in cat_cols:
        print(f"\nVariable: {col}")
        print(f"Categorías únicas: {df[col].nunique()}")
        print(df[col].value_counts().head(10))

### *7.2.2 Comportamiento de las variables categoricas*

In [ ]:
def visualizar_variables_categoricas(df):
    """
    Muestra gráficos de barras para todas las variables categóricas.
    """
    cat_cols = df.select_dtypes(include=['object']).columns.tolist()
    num_cols = len(cat_cols)
    cols = 3
    rows = (num_cols // cols) + (num_cols % cols > 0)
    fig, axes = plt.subplots(rows, cols, figsize=(15, 5*rows))
    axes = axes.flatten()
    for i, col in enumerate(cat_cols):
        sns.countplot(data=df, x=col, ax=axes[i], order=df[col].value_counts().index)
        axes[i].set_title(col)
        axes[i].tick_params(axis='x', rotation=45)
    for j in range(i + 1, len(axes)):
        axes[j].axis('off')
    plt.tight_layout()
    plt.show()

### *7.2.3 Comportamiento de las variables categoricas vs variable objetivo*

In [ ]:
def graficar_categoricas_vs_objetivo(df, var_objetivo):
    """
    Genera countplots para cada variable categórica contra la variable objetivo.
    """
    cat_cols = df.select_dtypes(include='object').columns
    for col in cat_cols:
        plt.figure(figsize=(8,4))
        sns.countplot(data=df, x=col, hue=var_objetivo, order=df[col].value_counts().index)
        plt.title(f"{col} vs {var_objetivo}")
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()

In [ ]:
graficar_categoricas_vs_objetivo(df, 'mental_health_risk')

### *7.2.4 Tabla de frecuencias cruzada*

In [ ]:
def tabla_frecuencia_cruzada(df, var_cat, var_objetivo):
    """
    Imprime tabla cruzada normalizada entre una variable categórica y la objetivo.
    """
    print(pd.crosstab(df[var_cat], df[var_objetivo], normalize='index'))

In [ ]:
tabla_frecuencia_cruzada(df, 'gender', 'mental_health_risk')